In [39]:
import numpy as np
import pandas as pd

import bokeh.layouts
from bokeh.io import curdoc, output_file, show
from bokeh.layouts import widgetbox
from bokeh.models import (
    BoxSelectTool,
    Circle, 
    ColumnDataSource,
    CustomJS,
    DataRange1d, 
    GMapPlot,
    GMapOptions,
    HoverTool,
    LogColorMapper,
    PanTool,
    Slider,
    WheelZoomTool,
)
from bokeh.models.glyphs import Patches
from bokeh.models.widgets import Button, RadioButtonGroup, Select, CheckboxButtonGroup
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

In [51]:
with open('NYC_boundries_cleaned.json') as f:
    data = json.load(f)
nyc_neighborhoods = data
len(nyc_neighborhoods)


195

# 3. Build Bokeh heat map

In [64]:
palette.reverse()

color_mapper = LogColorMapper(palette=palette)

neighborhood_xs = [neighborhood["Lon"] for neighborhood in nyc_neighborhoods.values()]
neighborhood_ys = [neighborhood["Lat"] for neighborhood in nyc_neighborhoods.values()]

neighborhood_names = list(nyc_neighborhoods.keys())

# Warning
# It is critical to note that no python code is ever executed when a CustomJS callback is used. 
# A CustomJS callback is only executed inside a browser JavaScript interpreter, 
# and can only directly interact JavaScript data and functions (e.g., BokehJS Backbone models).
#
# Thoughts: 
# Prepare the Uber pickup counts data, save them in to a matrix
# Dimensions: row number = month# * day#  * hour#  = 3*31*24 = 2232
#            column # = 195
#


neighborhood_counts =np.zeros(195)
for i in range(1, 744*3):
    append_row = np.zeros(195)+i
    neighborhood_counts = np.vstack((neighborhood_counts, append_row))

neighborhood_counts_uber = neighborhood_counts.tolist()
neighborhood_counts_taxi = neighborhood_counts.tolist()

# All 195 columns with given input value
months = np.zeros(len(nyc_neighborhoods), dtype=int)+4

days = np.zeros(len(nyc_neighborhoods), dtype=int)+1

hours = np.zeros(len(nyc_neighborhoods), dtype=int)
indicators = np.zeros(len(nyc_neighborhoods), dtype=int)

source_uber = ColumnDataSource(data=dict(
    x = neighborhood_xs,
    y = neighborhood_ys,
    name = neighborhood_names,
    
    month = months,
    day = days,
    hour = hours,
    
    values_uber = neighborhood_counts_uber,
    value_uber = neighborhood_counts_uber[0]
    
    values_taxi = neighborhood_counts_taxi,
    value_taxi = neighborhood_counts_taxi[0]
    
    indicator = indicators
    
    
    # 0-based
    # One month: 31*24 = 744 rows
    # One day: 24 rows
    #
    
    # April: row 0-743 (0:744)
        # Day: chunck size 31
            # hour: chunck size 24
    # May: row 744-1487 (744:1488)
    # Jun: row 1488-end
    
    # Example:month 5 day 12 hour 18 
    # index = 744*($5$-4) + 24*($12$-1) + $18$
    
    # value = neighborhood_counts[744*(min(months)-4) + 24*(min(days)-1) + min(hours)]
    
))

/Users/zicong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/Users/zicong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


In [73]:
callback_month = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var month = month.value;
    
    d = data['day'][0]
    h = data['hour'][0]
    
    
    for (i = 0; i < data['month'].length; i++) {
        data['month'][i] = month
    }
    
    values = data['values']
    data['value'] = values[744*(month-4) + 24*(d-1) + h]
    
    source.trigger('change');
""")

callback_day = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var day = day.value;
    
    m = data['month'][0]
    h = data['hour'][0]

    for (i = 0; i < data['day'].length; i++) {
        data['day'][i] = day
    }
    
    values = data['values']
    data['value'] = values[744*(m-4) + 24*(day-1) + h]

    source.trigger('change');
""")

callback_hour = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var hour = hour.value;
    var indicator = indicator.value
    
    m = data['month'][0]
    d = data['day'][0]
    
    for (i = 0; i < data['hour'].length; i++) {
        data['hour'][i] = hour
    }
    
    values = data['values']
    data['value'] = values[744*(m-4) + 24*(d-1) + hour]
    
    source.trigger('change');
""")


callback_indicator = CustomJS(args=dict(source=source), code="""
    var indicator = indicator.value
    console.log(indicator)

    if (indicator[0] == 0) {
        var data = source.data;
        var hour = hour.value;

        m = data['month'][0]
        d = data['day'][0]
        h = data['hour'][0]

        values = data['values_uber']
        data['value_uber'] = values[744*(m-4) + 24*(d-1) + h]

        source.trigger('change');
        
    } else {
        var data = source.data;
        var hour = hour.value;

        m = data['month'][0]
        d = data['day'][0]
        h = data['hour'][0]

        values = data['values_taxi']
        data['value_taxi'] = values[744*(m-4) + 24*(d-1) + h]*2

        source.trigger('change'); 
""")


In [78]:
TOOLS = "pan,wheel_zoom,reset,hover,save"

plot = figure(title="Uber Pickup Distribution", tools=TOOLS, 
              x_axis_location=None, y_axis_location=None, plot_width=300)
plot.plot_height = 600

plot.grid.grid_line_color = None

plot.patches('x', 'y', source=source,
              fill_color={'field': 'value', 'transform': color_mapper},
              fill_alpha=0.7, line_color="black", line_width=0.5)

hover = plot.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("(Long, Lat)", "($x, $y)"),
    ("Number of pickups", "@value"),
    ("Month", "@month"),
    ("Day", "@day"),
    ("Hour", "@hour")
]

map_select = Select(title="Map:", value="map", options=['0', '1'], callback=callback_indicator)
callback_indicator.args["indicator"] = map_select

hour_slider = Slider(start=0, end=23, step=1, value=0, title="Hour", callback=callback_hour)
callback_hour.args["hour"] = hour_slider

month_select = Select(title="Month:", value="Month", options=['4', '5', '6'], callback=callback_month)
callback_month.args["month"] = month_select
                                        
days = [str(i) for i in list(range(1,32))]
day_select = Select(title="Day:", value="Day", options=days, callback=callback_day)
callback_day.args["day"] = day_select

In [76]:
# from bokeh.charts import Dot, show, output_file
# data = {
#     'sample': ['1st', '2nd', '1st', '2nd', '1st', '2nd'],
#     'interpreter': ['python', 'python', 'pypy', 'pypy', 'jython', 'jython'],
#     'timing': [-2, 5, 12, 40, 22, 30],
# }

# # x-axis labels pulled from the interpreter column, stacking labels from sample column
# dots = Dot(data, values='timing', label='interpreter',
#            group='sample', agg='mean',
#            title="Python Interpreter Sampling",
#            legend='top_right', width=600)

# curdoc().clear()
# show(dots)

In [79]:
curdoc().clear()
show(bokeh.layouts.row(plot, checkbox_button_group, widgetbox(hour_slider, month_select, day_select, width=300)))